In [35]:
import cv2
import os
from skimage.metrics import structural_similarity
import numpy as np

In [36]:
# video_dir='F:/Sketch_Detection/ActionNet approach/test_videos'
# video_name='S2_T10_Ambady.mp4' 
# video_name = os.path.join(video_dir,video_name)
# cap = cv2.VideoCapture(video_name)
# dataset_path= 'F:/Sketch_Detection/ActionNet approach/action_data'
# train_size = 299

In [37]:
# fps = cap.get(cv2.CAP_PROP_FPS)
# width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
# height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
# frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [38]:
frameA = cv2.imread('frame960.jpg')
frameB = cv2.imread('frame1080.jpg')
train_size=299

In [39]:
def compare_frame(frameA, frameB):
  grayA = cv2.cvtColor(frameA, cv2.COLOR_BGR2GRAY)
  grayB = cv2.cvtColor(frameB, cv2.COLOR_BGR2GRAY)
   
  score, diff = structural_similarity(grayA, grayB, full=True)
  diff = (diff * 255).astype("uint8")
  
  print("SSIM: {}".format(score))

  thresh = cv2.threshold(diff, 180, 255, cv2.THRESH_BINARY_INV)[1]
  cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
  cnts=cnts[0]

  return diff, thresh, cnts, score

def convert_box(cnts):
  box = []
  for c in cnts:
    x, y, w, h = cv2.boundingRect(c)
    if w > 5 and h > 16: 
      box.append([x, y, x+w, y+h])
  box = np.array(box)
  print(box)
  return box

def non_max_suppression(boxes, overlapThresh):
  if len(boxes) == 0:
    return []
  if boxes.dtype.kind == "i":
    boxes = boxes.astype("float")

  pick = []
  x1 = boxes[:,0]
  y1 = boxes[:,1]
  x2 = boxes[:,2]
  y2 = boxes[:,3]
  area = (x2 - x1 + 1) * (y2 - y1 + 1)
  idxs = np.argsort(y2)

  while len(idxs) > 0:
   last = len(idxs) - 1
   i = idxs[last]
   pick.append(i)
   xx1 = np.maximum(x1[i], x1[idxs[:last]])
   yy1 = np.maximum(y1[i], y1[idxs[:last]])
   xx2 = np.minimum(x2[i], x2[idxs[:last]])
   yy2 = np.minimum(y2[i], y2[idxs[:last]])
   w = np.maximum(0, xx2 - xx1 + 1)
   h = np.maximum(0, yy2 - yy1 + 1)
   overlap = (w * h) / area[idxs[:last]] 
   idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))

  return boxes[pick].astype("int")
  

def find_max(boxes_nms):
  if len(boxes_nms) == 0:
    return []
  boxes = []
  for box_nms in boxes_nms:
    box_nms = np.append(box_nms, (box_nms[2]-box_nms[0])*(box_nms[3]-box_nms[1]))
    boxes.append(box_nms)
  boxes = np.array(boxes)
  idx = np.argsort(boxes[:,4])
  x_center = boxes[idx[-1]][0] + (boxes[idx[-1]][2] - boxes[idx[-1]][0]) / 2
  y_center = boxes[idx[-1]][1] + (boxes[idx[-1]][3] - boxes[idx[-1]][1]) / 2
  box_max = np.append(boxes[idx[-1]], [x_center, y_center])
  box_max = np.array(box_max, dtype = np.int32)
  return box_max

def find_max_region(boxes_nms):
  if len(boxes_nms) == 0:
    return []
  x1 = boxes_nms[:,0]
  y1 = boxes_nms[:,1]
  x2 = boxes_nms[:,2]
  y2 = boxes_nms[:,3]  
  xx1 = min(x1)
  yy1 = min(y1)
  xx2 = max(x2)
  yy2 = max(y2)
  max_region = np.array([xx1, yy1, xx2, yy2])
  print(max_region)
  return max_region

def to_canvas(region):
  canvas = np.zeros((train_size, train_size, 3), np.uint8)
  if region.shape[0] > region.shape[1]:
    if region.shape[1] % 2:
      canvas[:, int(train_size / 2 - (region.shape[1] / 2)):int(train_size / 2 + (region.shape[1] / 2) + 1)] = region
    else:
      canvas[:, int(train_size / 2 - (region.shape[1] / 2)):int(train_size / 2 + (region.shape[1] / 2))] = region
  else:
    if region.shape[0] % 2:
      canvas[int(train_size / 2 - (region.shape[0] / 2)):int(train_size / 2 + (region.shape[0] / 2) + 1), :] = region
    else:
      canvas[int(train_size / 2 - (region.shape[0] / 2)):int(train_size / 2 + (region.shape[0] / 2)), :] = region
  return canvas

In [40]:
def change_region():

 diff, thresh, cnts, score= compare_frame(frameA, frameB)
 mask_region = np.zeros((frameB.shape[0], frameB.shape[1]), np.uint8)
 boxes = convert_box(cnts)
 boxes_nms = non_max_suppression(boxes, 0.3)
 max_box = find_max(boxes_nms)
 max_region_box = find_max_region(boxes_nms)
 for box in boxes_nms:
    cv2.rectangle(mask_region, (box[0], box[1]), (box[2], box[3]), (255, 255, 255), -1)

 frameC = frameB.copy()
 frameD = frameB.copy()

 if len(boxes_nms):
    cv2.rectangle(frameC, (max_region_box[0], max_region_box[1]), (max_region_box[2], max_region_box[3]), (0, 255, 0), 2)
    max_region_A = frameA[max_region_box[1]:max_region_box[3], max_region_box[0]:max_region_box[2]].copy()
    mask_region = mask_region[max_region_box[1]:max_region_box[3], max_region_box[0]:max_region_box[2]].copy()
    max_region_D = frameD[max_region_box[1]:max_region_box[3], max_region_box[0]:max_region_box[2]].copy()
    region_A = cv2.bitwise_and(max_region_A, max_region_A, mask = mask_region)
    region_D = cv2.bitwise_and(max_region_D, max_region_D, mask = mask_region) 
    f_rate =  train_size* 1.0 / max(region_A.shape[0], region_A.shape[1])
    region_A = cv2.resize(region_A, (0,0), fx = f_rate, fy = f_rate) 
    region_D = cv2.resize(region_D, (0,0), fx = f_rate, fy = f_rate) 
    img_A = to_canvas(region_A)
    img_D = to_canvas(region_D)
    
    cv2.imshow('img_A',img_A)
    cv2.imshow('img_D',img_D)
    cv2.imshow('frame_A',frameA)
    cv2.imshow('frame_B',frameB)
    cv2.imshow('region_A',region_A)
    cv2.imshow('region_B',region_D)
    cv2.waitKey(0)
    


SSIM: 0.9894188102547747
[[ 64 120 281 400]]
[   64   120   281   400 60760   172   260]
[ 64 120 281 400]
